In [ ]:
!pip install mlxtend
import pandas as pd
from itertools import combinations
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import fpgrowth
import time


In [5]:
# Define product lists for different stores: Walmart, Target, and Costco
walmart_items = [
    "Headphones", "Smart TV", "Tablet", "Fitness Tracker",
    "Bluetooth Speaker", "E-Reader", "USB Cable", "Smart Light Bulb",
    "External SSD", "Video Game Console"
]

target_items = [
    "Bedding", "Kitchen Appliances", "Cookware",
    "Bath Towels", "Wall Art", "Laundry Hamper",
    "Storage Bins", "Garden Supplies", "Air Fryer",
    "Coffee Machine"
]

costco_items = [
    "Television", "Gaming Console", "Laptop", "Wi-Fi Router",
    "Noise-Canceling Headphones", "Smartwatch", "Portable Speaker",
    "SSD Drive", "Home Theater", "Video Doorbell"
]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# Transactions for each store
store_transactions = {
    "Walmart": [
        ["Headphones", "Fitness Tracker", "Bluetooth Speaker"],
        ["Smart TV", "Tablet"],
        ["Smart Light Bulb", "External SSD"],
        ["Video Game Console", "Headphones", "Fitness Tracker"],
        ["Bluetooth Speaker", "Smart TV"],
        ["Tablet", "USB Cable"],
        ["Headphones", "E-Reader"],
        ["Fitness Tracker", "Video Game Console"],
        ["Smart Light Bulb", "Bluetooth Speaker"],
        ["External SSD", "Tablet"]
    ],
    "Target": [
        ["Bedding", "Kitchen Appliances", "Bath Towels"],
        ["Cookware", "Air Fryer"],
        ["Wall Art", "Laundry Hamper"],
        ["Storage Bins", "Garden Supplies", "Coffee Machine"],
        ["Bedding", "Laundry Hamper"],
        ["Kitchen Appliances", "Bath Towels"],
        ["Air Fryer", "Storage Bins"],
        ["Coffee Machine", "Wall Art"],
        ["Garden Supplies", "Bedding"],
        ["Bath Towels", "Kitchen Appliances"]
    ],
    "Costco": [
        ["Television", "Gaming Console", "Wi-Fi Router"],
        ["Laptop", "Noise-Canceling Headphones"],
        ["Smartwatch", "Portable Speaker"],
        ["SSD Drive", "Home Theater", "Video Doorbell"],
        ["Television", "Laptop"],
        ["Wi-Fi Router", "Smartwatch"],
        ["Noise-Canceling Headphones", "Gaming Console"],
        ["Portable Speaker", "Video Doorbell"],
        ["SSD Drive", "Home Theater"],
        ["Smartwatch", "Gaming Console"]
    ]
}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
# Get user input for minimum support and confidence
min_support = float(input("Enter minimum support  "))
min_confidence = float(input("Enter minimum confidence  "))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Enter minimum support (e.g., 0.2): 0.2
Enter minimum confidence (e.g., 0.7): 0.7


In [9]:
# Convert transactions into a binary-encoded DataFrame for algorithms
def convert_transactions(transactions, products):
    encoded_data = []
    for transaction in transactions:
        encoded_transaction = {product: (product in transaction) for product in products}
        encoded_data.append(encoded_transaction)
    return pd.DataFrame(encoded_data)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:


# Exhaustive Search Implementation
def exhaustive_search(transactions, products, min_support):
    start = time.time()

    # Helper function to compute support for a given itemset
    def support_calculation(itemset, transactions):
        count = 0
        for transaction in transactions:
            if itemset.issubset(transaction):
                count += 1
        return count / len(transactions)

    # Helper function to find frequent itemsets of a certain size
    def find_frequent_itemsets(products, transactions, size, min_support):
        itemsets = [set(combo) for combo in combinations(products, size)]
        frequent_itemsets = []
        for itemset in itemsets:
            support = support_calculation(itemset, transactions)
            if support >= min_support:
                frequent_itemsets.append((itemset, support))
        return frequent_itemsets

    # Start with single itemsets and increase size iteratively
    k = 1
    frequent_itemsets = []
    while True:
        current_itemsets = find_frequent_itemsets(products, transactions, k, min_support)
        if not current_itemsets:
            break
        frequent_itemsets.extend(current_itemsets)
        k += 1

    # Generate association rules from frequent itemsets
    rules = []
    for itemset, support in frequent_itemsets:
        if len(itemset) > 1:
            for consequent in itemset:
                antecedent = itemset - {consequent}
                if antecedent:
                    confidence = support_calculation(itemset, transactions) / support_calculation(antecedent, transactions)
                    if confidence >= min_confidence:
                        rules.append((list(antecedent), list(consequent), confidence, support))

    total_time = time.time() - start
    return frequent_itemsets, rules, total_time

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# Apriori Algorithm
def apriori_method(df, min_support, min_confidence):
    start = time.time()
    frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x))
    rules['consequents'] = rules['consequents'].apply(lambda x: list(x))
    apriori_time = time.time() - start
    return frequent_itemsets, rules, apriori_time

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# FP-Growth Algorithm
def fpgrowth_method(df, min_support, min_confidence):
    start = time.time()
    frequent_itemsets = fpgrowth(df, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x))
    rules['consequents'] = rules['consequents'].apply(lambda x: list(x))
    fpgrowth_time = time.time() - start
    return frequent_itemsets, rules, fpgrowth_time

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# Function to compare algorithms
def compare_methods(store_transactions, store_items, min_support, min_confidence):
    for store, transactions in store_transactions.items():
        print(f"\n--- {store} ---")
        products = store_items[store]
        transactions = [set(t) for t in transactions]

        # Exhaustive Search
        frequent_itemsets_es, rules_es, es_time = exhaustive_search(transactions, products, min_support)
        print(f"Exhaustive Search Time: {es_time:.4f} seconds")
        print(f"Frequent Itemsets (Exhaustive Search): {frequent_itemsets_es}")

        if rules_es:
            print("\nAssociation Rules (Exhaustive Search):")
            for idx, (antecedent, consequent, confidence, support) in enumerate(rules_es, start=1):
                print(f"Rule {idx}: {antecedent} -> {consequent}")
                print(f"Confidence: {confidence * 100:.2f}%")
                print(f"Support: {support * 100:.2f}%\n")

        # Apriori
        df_encoded = convert_transactions(transactions, products)
        frequent_itemsets_ap, rules_ap, ap_time = apriori_method(df_encoded, min_support, min_confidence)
        print(f"Apriori Time: {ap_time:.4f} seconds")
        print(f"Frequent Itemsets (Apriori):\n{frequent_itemsets_ap}")

        if not rules_ap.empty:
            print("\nAssociation Rules (Apriori):")
            for idx, row in rules_ap.iterrows():
                print(f"Rule {idx + 1}: {row['antecedents']} -> {row['consequents']}")
                print(f"Confidence: {row['confidence'] * 100:.2f}%")
                print(f"Support: {row['support'] * 100:.2f}%\n")

        # FP-Growth
        frequent_itemsets_fp, rules_fp, fp_time = fpgrowth_method(df_encoded, min_support, min_confidence)
        print(f"FP-Growth Time: {fp_time:.4f} seconds")
        print(f"Frequent Itemsets (FP-Growth):\n{frequent_itemsets_fp}")

        if not rules_fp.empty:
            print("\nAssociation Rules (FP-Growth):")
            for idx, row in rules_fp.iterrows():
                print(f"Rule {idx + 1}: {row['antecedents']} -> {row['consequents']}")
                print(f"Confidence: {row['confidence'] * 100:.2f}%")
                print(f"Support: {row['support'] * 100:.2f}%\n")

# Dictionary mapping stores to their products
store_items = {
    "Walmart": walmart_items,
    "Target": target_items,
    "Costco": costco_items
}

# Run the comparison for Walmart, Target, and Costco
compare_methods(store_transactions, store_items, min_support, min_confidence)


--- Walmart ---
Exhaustive Search Time: 0.0005 seconds
Frequent Itemsets (Exhaustive Search): [({'Headphones'}, 0.3), ({'Smart TV'}, 0.2), ({'Tablet'}, 0.3), ({'Fitness Tracker'}, 0.3), ({'Bluetooth Speaker'}, 0.3), ({'Smart Light Bulb'}, 0.2), ({'External SSD'}, 0.2), ({'Video Game Console'}, 0.2), ({'Fitness Tracker', 'Headphones'}, 0.2), ({'Fitness Tracker', 'Video Game Console'}, 0.2)]

Association Rules (Exhaustive Search):
Rule 1: ['Video Game Console'] -> ['F', 'i', 't', 'n', 'e', 's', 's', ' ', 'T', 'r', 'a', 'c', 'k', 'e', 'r']
Confidence: 100.00%
Support: 20.00%

Apriori Time: 0.0182 seconds
Frequent Itemsets (Apriori):
   support                               itemsets
0      0.3                           (Headphones)
1      0.2                             (Smart TV)
2      0.3                               (Tablet)
3      0.3                      (Fitness Tracker)
4      0.3                    (Bluetooth Speaker)
5      0.2                     (Smart Light Bulb)
6      0.2 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
